In [23]:
!pip install joblib

In [24]:
import joblib

# Load your existing large model
clf_model = joblib.load("../models/classifier.pkl")

# Compress and save the loaded model
joblib.dump(clf_model, "classifier_new.pkl", compress=9)

print("Model compressed and saved as classifier_new.pkl")

Model compressed and saved as classifier_new.pkl


In [25]:
import joblib

clf_model = joblib.load("../models/classifier.pkl")

# Extra attributes delete kardo jo model ke liye zaruri nahi hain
if hasattr(clf_model, 'n_features_in_'):
    print("Model features:", clf_model.n_features_in_)

# Save with compression (already doing)
joblib.dump(clf_model, "classifier_new.pkl", compress=('xz', 9))


Model features: 10


['classifier_new.pkl']

In [26]:
import os
print(os.listdir("../models"))


['classifier.pkl', 'classifier_new.pkl', 'regressor.pkl']


In [27]:
import joblib

# Load Trained Models
clf_model = joblib.load("classifier_new.pkl")
reg_model = joblib.load("../models/regressor.pkl")


In [28]:
print(reg_model.named_steps['preprocessor'].get_feature_names_out())


['num__Age' 'num__Annual_Income' 'num__Family_Size' 'num__Credit_Score'
 'num__Travel_Frequency' 'num__Web_Visits' 'num__Email_Engagement'
 'cat__Gender_Female' 'cat__Gender_Male' 'cat__Marital_Status_Divorced'
 'cat__Marital_Status_Married' 'cat__Marital_Status_Single'
 'cat__Previous_Package_Basic' 'cat__Previous_Package_Deluxe'
 'cat__Previous_Package_King' 'cat__Previous_Package_Standard'
 'cat__Previous_Package_Super Deluxe' 'cat__Previous_Package_nan']


In [29]:
import pandas as pd
import joblib

In [31]:
#  Load Trained Models
clf_model = joblib.load("classifier_new.pkl")
reg_model = joblib.load("../models/regressor.pkl")

In [32]:

#  Create New Customer Data
new_customers = pd.DataFrame([
    {
        "age": 28,
        "annual_income": 85000,
        "family_size": 2,
        "credit_score": 680,
        "travel_frequency": 3,
        "web_visits": 14,
        "email_engagement": 8,
        "gender": "female",
        "marital_status": "married",
        "previous_package": "deluxe"
    },
    {
        "age": 45,
        "annual_income": 60000,
        "family_size": 4,
        "credit_score": 720,
        "travel_frequency": 1,
        "web_visits": 5,
        "email_engagement": 2,
        "gender": "male",
        "marital_status": "single",
        "previous_package": "basic"
    }
])


In [33]:
# Rename columns to match training format
new_customers.rename(columns={
    "age": "Age",
    "annual_income": "Annual_Income",
    "family_size": "Family_Size",
    "credit_score": "Credit_Score",
    "travel_frequency": "Travel_Frequency",
    "web_visits": "Web_Visits",
    "email_engagement": "Email_Engagement",
    "gender": "Gender",
    "marital_status": "Marital_Status",
    "previous_package": "Previous_Package"
}, inplace=True)


In [34]:
new_customers

,Age,Annual_Income,Family_Size,Credit_Score,Travel_Frequency,Web_Visits,Email_Engagement,Gender,Marital_Status,Previous_Package
0,28,85000,2,680,3,14,8,female,married,deluxe
1,45,60000,4,720,1,5,2,male,single,basic


In [35]:
# Make Predictions
# Predict if customer will purchase (0 or 1)
purchase_preds = clf_model.predict(new_customers)
purchase_probs = clf_model.predict_proba(new_customers)[:, 1]  

In [36]:
print(reg_model.named_steps['preprocessor'].feature_names_in_)


['Age' 'Gender' 'Marital_Status' 'Annual_Income' 'Family_Size'
 'Credit_Score' 'Previous_Package' 'Travel_Frequency' 'Web_Visits'
 'Email_Engagement']


In [37]:
print(new_customers.columns)


Index(['Age', 'Annual_Income', 'Family_Size', 'Credit_Score',
       'Travel_Frequency', 'Web_Visits', 'Email_Engagement', 'Gender',
       'Marital_Status', 'Previous_Package'],
      dtype='object')


In [38]:
def predict_for_new_customers(new_customers):
    # ✅ 1. Convert input into DataFrame
    df_new = pd.DataFrame(new_customers if isinstance(new_customers, list) else [new_customers])
    
    # ✅ 2. Clean column names
    df_new.columns = df_new.columns.str.lower()
    
    # ✅ 3. Run predictions
    purchase_preds = clf_model.predict(df_new)
    purchase_probs = clf_model.predict_proba(df_new)[:, 1]
    probability_scores = reg_model.predict(df_new)
    
    # ✅ 4. Merge results
    df_new["will_purchase"] = purchase_preds
    df_new["purchase_probability"] = purchase_probs.round(3)
    df_new["predicted_probability_score"] = probability_scores.round(3)
    
    return df_new
